In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/home/kunmok/Git/chisel-bootcamp/source/load-ivy.sc"

In [2]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

In [3]:
class MyOperators extends Module {
  val io = IO(new Bundle {
    val in      = Input(UInt(2.W))
    val out= Decoupled(Vec(3, UInt(2.W)))
  })
    val shiftReg = RegInit(VecInit(Seq.fill(3)(0.U(2.W))))
    val sReg = RegInit(VecInit(Seq.fill(3)(0.U(1.W)))) // initialze memory with all zeros
    val outValReg = RegInit(0.U(1.W))
    val cnt = RegInit(0.U(3.W))
    val one = 1.U(1.W)
    
    cnt := cnt + 1.U
    when(cnt === 2.U){
        shiftReg(0) := 1.U 
        for (n <- 1 to 2){
            shiftReg(n) := 1.U
        }   
        shiftReg(0) := 2.U 
    }.otherwise{
        shiftReg(0) := io.in
        (1 to 2).reverse.map(i => {shiftReg(i) := shiftReg(i-1)})
    }
    
    io.out.bits := shiftReg 
    io.out.valid := outValReg 
    
}

class MyOperatorsTester(c: MyOperators) extends PeekPokeTester(c) {
  poke(c.io.in, 3)  //cnt == 0.U
  step(1)  //cnt == 1 
    expect(c.io.out.bits(0), 3)
    expect(c.io.out.bits(1), 0)
    expect(c.io.out.bits(2), 0)
  step(1) //cnt == 2
    expect(c.io.out.bits(0), 3)
    expect(c.io.out.bits(1), 3)
    expect(c.io.out.bits(2), 0)
  step(1) // cnt == 3
    expect(c.io.out.bits(0), 2)
    expect(c.io.out.bits(1), 1)
    expect(c.io.out.bits(2), 1)

}
assert(Driver(() => new MyOperators) {c => new MyOperatorsTester(c)})
println("SUCCESS!!")

[info] [0.003] Elaborating design...
[info] [0.238] Done elaborating.
Total FIRRTL Compile Time: 347.1 ms
Total FIRRTL Compile Time: 36.1 ms
End of dependency graph
Circuit state created
[info] [0.003] SEED 1540973766062
test cmd2WrapperHelperMyOperators Success: 9 tests passed in 8 cycles taking 0.036103 seconds
[info] [0.015] RAN 3 CYCLES PASSED
SUCCESS!!


defined class MyOperators
defined class MyOperatorsTester